In [1]:
import os, sys
sys.path.append(os.path.join(os.getcwd(), ".."))
main_dir = os.path.abspath('..')
os.chdir(main_dir)
sys.path.append(main_dir)

import numpy as np
import sympy as sp
import re, copy
from sklearn.model_selection import train_test_split
from PhysicsRegression import PhyReg

### Step1: load Plasma Pressure data

The Plasma Pressure Data contains 12 datasets of different Kp (Kp=0,1,2,3,4,5) and Pd (Pd=1.5,3.0).
We first transform the data from Cartesian coordinates to polar coordinates, using the data with r<28 for training.

In [2]:
x_to_fit = []
y_to_fit = []
units = []

for Kp in ["0", "1", "2", "3", "4", "5"]:
    for Pd in ["1.5", "3.0"]:
        with open(f"./physical/data/TH-Gt_beta-ge-0.1_Kp={Kp}_Pd={Pd}_50-quartile.txt", "r") as fi:
            context = [c.split() for c in fi.read().split("\n")][1:-1]

        x = np.array([
            [float(c[0]) for c in context],
            [float(c[1]) for c in context]
        ]).transpose()
        y = np.array([float(c[6]) for c in context]).reshape((-1, 1))

        idx_nan = (y < 1e-10).reshape((-1))
        x = x[~idx_nan]
        y = y[~idx_nan]

        idx_pos = (x[:, 0] > 0).reshape((-1))
        x = x[~idx_pos]
        y = y[~idx_pos]

        x[:,0] = - x[:,0]                           #should be [0, 40]
        x[:,1] = x[:,1]                             #should be [-20, 20]

        x_polar = np.array([
            np.sqrt(x[:,0]**2 + x[:,1]**2) / 8,     #should be [0, 5]
            np.arctan2(x[:,1], x[:,0])              #should be [-pi/2, pi/2]
        ]).transpose()

        x_polar[:,1] = x_polar[:,1] + np.pi / 2     #should be [0, pi]
        
        assert all(0 <= x_polar[:,0]) and all(x_polar[:,0] <= 10)
        assert all(0 <= x_polar[:,1]) and all(x_polar[:,1] <= np.pi)

        x_to_fit.append(x_polar)
        y_to_fit.append(y)

for i in range(len(x_to_fit)):
    x_train, x_test, y_train, y_test = train_test_split(x_to_fit[i], y_to_fit[i], test_size=0.1, random_state=2024)
    x_to_fit[i] = x_train
    y_to_fit[i] = y_train

_x_to_fit = []
_y_to_fit = []
for j in [3.5]:
    for i in range(len(x_to_fit)):
        train_idx = x_to_fit[i][:, 0] < j
        x_train = x_to_fit[i][train_idx]
        y_train = y_to_fit[i][train_idx]
        _x_to_fit.append(x_train)
        _y_to_fit.append(y_train)
x_to_fit = _x_to_fit
y_to_fit = _y_to_fit

### Step2: inference with PhyReg

In [3]:
phyreg = PhyReg(
    path = "./model.pt",
    max_len=1000,
)

phyreg.fit(
    x_to_fit, y_to_fit, 
    use_Divide=True, 
    use_MCTS=False, 
    use_GP=False, 
    use_pysr_init=True, 
    use_const_optimization=False,
    verbose=False,
    oracle_name="physical2",
    oracle_file="./physical/data/oracle_model_case2/",
    oracle_bs=64, oracle_lr=0.004, oracle_epoch=1000,
    use_seperate_type=["id"],
    save_oracle_model=True
)

### Step3: Optimize constants

The symbolic formulas of 12 datasets are not the same, we hope to extract a uniform symbolic formula and optimize the constant at the same times.

In [ ]:
num_bfgs = 60

current_refined_exprs = []      # 12 list (12 formulas)
current_refined_mses = []       # 12 mses

for i in range(12):

    current_refined_expr = []   # 12 formulas
    current_refined_mse = 0     # 1 mses

    for j in range(12):

        best_expr = ""
        best_mse = 1e10

        expr = str(phyreg.best_gens[i]["predicted_tree"])
        for k,v in {
            "add": "+", "mul": "*", "sub": "-", "pow": "**", "inv": "1/", "neg": "-"
        }.items():
            expr = expr.replace(k, v)

        consts = re.findall(r"[+-]?\b\d+\.\d+\b", expr)
        for t,c in enumerate(consts):
            expr = expr.replace(c, f"c_{t}", 1)
        consts = np.array(list(map(float, consts)))

        x = x_to_fit[j]; y = y_to_fit[j]

        for trial in range(num_bfgs):

            new_consts = copy.deepcopy(consts)
            new_consts = new_consts + np.random.normal(0, 1, new_consts.shape)
            new_expr = copy.deepcopy(expr)

            new_consts = phyreg._const_optimize(f"log({new_expr})", (x, np.log(y)), len(consts), new_consts)
            for t,c in enumerate(new_consts):
                new_expr = new_expr.replace(f"c_{t}", str(c), 1)
            
            x_variables = [sp.Symbol('x_{}'.format(t)) for t in range(2)]
            f = sp.lambdify(x_variables, new_expr, "numpy")
            pred = f(*x.T)
            mse = phyreg.eval_metric(np.log(y), np.log(pred), metric="mse")

            if mse < best_mse:
                best_mse = mse
                best_expr = new_expr
    
        current_refined_expr.append(best_expr)
        current_refined_mse += best_mse

    current_refined_exprs.append(current_refined_expr)
    current_refined_mses.append(current_refined_mse)

idx = np.argmin(current_refined_mses)
refined_exprs = current_refined_exprs[idx]

phyreg.express_skeleton(
    [{"predicted_tree":refined_exprs[i]} for i in range(len(refined_exprs))],
    use_sp=True
)

idx          : 0
expr skeleton: C_0 + C_1/((x_0 - C_2)**2*(cos(C_3*x_1) + C_4))
constants    : 0.0368 0.00785 0.164 0.0153 -0.991

idx          : 1
expr skeleton: C_0 + C_1/((x_0 - C_2)**2*(cos(C_3*x_1) + C_4))
constants    : 0.0617 0.00191 0.0833 0.00705 -0.998

idx          : 2
expr skeleton: C_0 + C_1/((x_0 - C_2)**2*(cos(C_3*x_1) + C_4))
constants    : 0.0280 64.047 0.0522 5.430 60.906

idx          : 3
expr skeleton: C_0 + C_1/((x_0 - C_2)**2*(cos(C_3*x_1) + C_4))
constants    : 0.0654 81.776 -0.0645 5.467 53.866

idx          : 4
expr skeleton: C_0 + C_1/((x_0 - C_2)**2*(cos(C_3*x_1) + C_4))
constants    : 0.0613 5.671 0.272 2.011 6.474

idx          : 5
expr skeleton: C_0 + C_1/((x_0 - C_2)**2*(cos(C_3*x_1) + C_4))
constants    : 0.108 70.007 0.196 3.507 59.886

idx          : 6
expr skeleton: C_0 + C_1/((x_0 - C_2)**2*(cos(C_3*x_1) + C_4))
constants    : 0.0459 30.013 0.191 3.496 29.325

idx          : 7
expr skeleton: C_0 + C_1/((x_0 - C_2)**2*(cos(C_3*x_1) + C_4))
constants  